<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: **SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Joaquín De Groote
- Nombre de alumno 2: Vicente Pinochet R.

### **Link de repositorio de GitHub:** [Insertar Enlace](https://github.com/Qajirr/MDS7202-Labs)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Inserte su código aquí
df_retail = pd.read_pickle('online_retail_data.pickle')
df_retail.head(5)

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [4]:
def explore_data(dataframe_in):
    # Histograma para Price
    fig_precio = px.histogram(dataframe_in,
                              x="Price",
                              nbins=20,
                              title="Distribución de Precios",
                              template='plotly_dark'
                              )
    fig_precio.show()

    # Histograma para Quantity
    fig_cantidad = px.histogram(dataframe_in,
                                x="Quantity",
                                nbins=20,
                                title="Distribución de Cantidad",
                                template='plotly_dark'
                                )
    fig_cantidad.show()
    # Nulos
    nulos = dataframe_in.isnull().sum()
    print("Conteo de datos nulos por variable:\n", nulos)

In [ ]:
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [6]:
class IQR(BaseEstimator, TransformerMixin):
  def __init__(self, lamb: float):
    #Completar código aquí
    self.lamb = lamb

  def fit(self, X):
    #Completar código aquí
    # Calcular los cuartiles Q1 y Q3
    self.q1 = X.quantile(0.25)
    self.q3 = X.quantile(0.75)
    # Rango intercuartil
    self.iqr = self.q3 - self.q1
    self.cola_inf = self.q1 - self.lamb * self.iqr
    self.cola_sup = self.q3 + self.lamb * self.iqr
    return self

  def transform(self, X):
    #Completar código aquí
    mask = (X >= self.cola_inf) & (X <= self.cola_sup)
    return X[mask]

  def set_output(self,transform='default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Price', 'Quantity']
categorical_columns = df_retail.columns.difference(numerical_columns).tolist()

# Definicion del pipeline
numeric_transformations = Pipeline([
    ('IQR', IQR(lamb=1.5))
])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)
                                        ],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

# Gráficos
explore_data(df_retail)
explore_data(df_iqr)


*Reporte los cambios observados aquí.*
**Cambios observados después de aplicar el pipeline con IQR:**

1. **Distribución de "Price" (Precio)**:
   - **Antes del IQR**: El histograma mostraba una **única columna grande**, indicando una alta concentración de datos en un rango pequeño, debido a la presencia de valores atípicos.
   - **Después del IQR**: La distribución de precios se vuelve más **visible**, con múltiples bins. Debido a que se eliminaron valores atípicos extremos que antes entorpecian la gráfica, permitiendo ver un mayor detalle en los valores comunes de precio.

2. **Distribución de "Quantity" (Cantidad)**:
   - **Antes del IQR**: Similar al precio, el histograma mostraba una única barra alta, indicando que la mayoría de los datos estaban agrupados, por a la presencia de valores atípicos.
   - **Después del IQR**: La distribución de la cantidad es más **detallada**, con varias barras visibles, lo que hace que los datos se distribuyan más uniformemente.

3. **Nulos antes y después de la transformación**:
   - **Antes del IQR**:
     - "Price": **8356** nulos.
     - "Quantity": **7915** nulos.
   - **Después del IQR**:
     - "Price": **40290** nulos.
     - "Quantity": **33698** nulos.
   - El aument en los valores nulos en ambas columnas es por la eliminación de los valores atípicos. Los valores fuera del rango intercuartil se eliminaron.

4. **Cambios relacionados con el ajuste de lambda**:
   - Al **aumentar el valor de lambda**, la distribución se va asemejando más a la original antes del pipeline. Esto se debe a que un valor más alto de lambda tiende a suavizar la transformación y puede reducir el efecto del IQR, permitiendo que algunos valores atípicos regresen a la visualización.


### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
# Escriba su respuesta aquí
from sklearn.impute import SimpleImputer
# Definicion del pipeline
categoric_transformations = Pipeline([
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))
])
# Modificar numeric_transformations
numeric_transformations = Pipeline([
    ('IQR', IQR(lamb=1.5)),
    ('mean_imputer', SimpleImputer(strategy='mean'))
])
# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)
                                        ],
                                       verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_mean_imputer = column_transformer.fit_transform(df_retail)
# Gráficos
explore_data(df_iqr)
explore_data(df_mean_imputer)

*Reporte los cambios observados aquí.*
Comparación entre `df_iqr` y `df_mean_imputer`

1. **Distribución de "Price" (Precio)**:
   - **Después de la imputación (media)**: La imputación con la media ha provocado un aumento significativo en el bin de valores **2 - 2.49**, que corresponde al promedio de los precios antes de la imputación. Este efecto es notable ya que **todos los valores nulos fueron reemplazados por el valor promedio**, lo que agrupa una mayor cantidad de datos en torno a este bin y puede distorsionar la percepción de la distribución original.

2. **Distribución de "Quantity" (Cantidad)**:
   - **Después de la imputación (media)**: El bin de valores **6 - 7** experimenta un aumento notable, ya que este era el promedio de las cantidades y todos los nulos se rellenaron con dicho valor. Esto crea una concentración alrededor de este rango.

3. **Conteo de Nulos antes y después de la imputación**:
   - **En IQR**:
     - **Price**: Existían **40,290** valores nulos antes de la imputación.
     - **Quantity**: Se contabilizaban **33,698** valores nulos.
   - **Después de la imputación (media)**:
     - **Price**: Los valores nulos fueron completamente eliminados **0** nulos.
     - **Quantity**: De manera similar, los valores nulos fueron reemplazados, quedando **0** nulos.
     - **Otras columnas**: Tampoco presentan valores nulos después de la imputación, ya que todas las columnas no numericas fueron procesadas por el `SimpleImputer`.

- En **IQR**, se observa una distribución más equilibrada y detallada tras la eliminación de valores atípicos, lo que permite analizar mejor las características de los datos.
- **Después de la imputación con la media**, los valores faltantes fueron reemplazados con el promedio, lo que resulta en concentraciones de datos alrededor de los promedios en las columnas numéricas, afectando la naturalidad de la distribución de los datos.


In [ ]:
# Escriba su respuesta aquí
from sklearn.impute import KNNImputer

# Modificar numeric_transformations nuevamente
numeric_transformations = Pipeline([
    ('IQR', IQR(lamb=1.5)),
    ('mean_imputer', KNNImputer(n_neighbors=3))
])
# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)
                                        ],
                                       verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_knn_imputer = column_transformer.fit_transform(df_retail)
# Gráficos
explore_data(df_iqr)
explore_data(df_knn_imputer)

*Reporte los cambios observados aquí.*
Comparación entre `df_iqr` y `df_knn_imputer`

1. **Distribución de "Price" (Precio)**:
   - **Después de la imputación (KNN)**: La imputación con KNN ha provocado un aumento en el los bin de valores **3.5 - 3.99**, que corresponde al promedio de los precios antes de la imputación. Este efecto es notable ya que **todos los valores nulos fueron reemplazados por el valor promedio**, lo que agrupa una mayor cantidad de datos en torno a este bin y puede distorsionar la percepción de la distribución original.

2. **Distribución de "Quantity" (Cantidad)**:
   - **Después de la imputación (KNN)**: El bin de valores **6 - 7** experimenta un aumento notable, ya que este era el promedio de las cantidades y todos los nulos se rellenaron con dicho valor. Esto crea una concentración alrededor de este rango.

3. **Conteo de Nulos antes y después de la imputación**:
   - **En IQR**:
     - **Price**: Existían **40,290** valores nulos antes de la imputación.
     - **Quantity**: Se contabilizaban **33,698** valores nulos.
   - **Después de la imputación (KNN)**:
     - **Price**: Los valores nulos fueron completamente eliminados **0** nulos.
     - **Quantity**: De manera similar, los valores nulos fueron reemplazados, quedando **0** nulos.
     - **Otras columnas**: Tampoco presentan valores nulos después de la imputación, ya que todas las columnas no numericas fueron procesadas por el `KNNImputer`.

- En **IQR**, se observa una distribución más equilibrada y detallada tras la eliminación de valores atípicos, lo que permite analizar mejor las características de los datos.
- **Después de la imputación con la KNN**, los valores faltantes fueron reemplazados con el promedio, lo que resulta en concentraciones de datos alrededor de los promedios en las columnas numéricas, afectando la naturalidad de la distribución de los datos.


In [ ]:
explore_data(df_mean_imputer)
explore_data(df_knn_imputer)

*Escriba su respuesta aquí*

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [31]:
def custom_features(dataframe_in):
    # Crear columna que contiene la primera fecha de compra del cliente
    first_purchase = dataframe_in.groupby('Customer ID')['InvoiceDate'].min().reset_index()
    first_purchase.columns = ['Customer ID', 'FirstPurchase']
    
    # Crear columna que contiene la última fecha de compra del cliente
    last_purchase = dataframe_in.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    last_purchase.columns = ['Customer ID', 'LastPurchase']
    
    # Unir las dos nuevas columnas al dataframe original
    df = pd.merge(dataframe_in, first_purchase, on='Customer ID', how='left')
    df = pd.merge(df, last_purchase, on='Customer ID', how='left')
    
    # Feature 1: Length (L)
    df['Length'] = (df['LastPurchase'] - df['FirstPurchase']).dt.days
    
    # Feature 2: Recency (R)
    # "Hoy" es la última fecha del dataset
    today = df['InvoiceDate'].max()
    df['Recency'] = (today - df['LastPurchase']).dt.days
    
    # Feature 3: Frequency (F)
    frequency = df.groupby('Customer ID')['Invoice'].nunique().reset_index()
    frequency.columns = ['Customer ID', 'Frequency']
    df = pd.merge(df, frequency, on='Customer ID', how='left')
    
    # Feature 4: Monetary (M)
    df['TotalPrice'] = df['Price'] * df['Quantity']
    monetary = df.groupby('Customer ID')['TotalPrice'].sum().reset_index()
    monetary['Monetary'] = monetary['TotalPrice'] / df.groupby('Customer ID')['Invoice'].nunique().reset_index()['Invoice']
    df = pd.merge(df, monetary[['Customer ID', 'Monetary']], on='Customer ID', how='left')
    
    # Feature 5: Periodicity (P)
    # Calcular las diferencias entre compras consecutivas
    df['InvoiceDateShift'] = df.groupby('Customer ID')['InvoiceDate'].shift(1)
    df['IVT'] = (df['InvoiceDate'] - df['InvoiceDateShift']).dt.days
    periodicity = df.groupby('Customer ID')['IVT'].std().reset_index()
    periodicity.columns = ['Customer ID', 'Periodicity']
    df = pd.merge(df, periodicity, on='Customer ID', how='left')
    
    # Seleccionamos las columnas que necesitamos
    result_df = df[['Customer ID', 'Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']].drop_duplicates(subset='Customer ID')
    
    return result_df

# Aplicar la función
df_retail = pd.read_pickle('online_retail_data.pickle')  # Asegúrate de cargar el archivo correctamente
df_lrmfp = custom_features(df_retail)

# Mostrar las primeras filas del resultado
df_lrmfp.head()

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
# Reutilizamos el custom_features definido previamente
def custom_features(dataframe_in):
    # Crear columna que contiene la primera fecha de compra del cliente
    first_purchase = dataframe_in.groupby('Customer ID')['InvoiceDate'].min().reset_index()
    first_purchase.columns = ['Customer ID', 'FirstPurchase']
    
    # Crear columna que contiene la última fecha de compra del cliente
    last_purchase = dataframe_in.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    last_purchase.columns = ['Customer ID', 'LastPurchase']
    
    # Unir las dos nuevas columnas al dataframe original
    df = pd.merge(dataframe_in, first_purchase, on='Customer ID', how='left')
    df = pd.merge(df, last_purchase, on='Customer ID', how='left')
    
    # Feature 1: Length (L)
    df['Length'] = (df['LastPurchase'] - df['FirstPurchase']).dt.days
    
    # Feature 2: Recency (R)
    today = df['InvoiceDate'].max()
    df['Recency'] = (today - df['LastPurchase']).dt.days
    
    # Feature 3: Frequency (F)
    frequency = df.groupby('Customer ID')['Invoice'].nunique().reset_index()
    frequency.columns = ['Customer ID', 'Frequency']
    df = pd.merge(df, frequency, on='Customer ID', how='left')
    
    # Feature 4: Monetary (M)
    df['TotalPrice'] = df['Price'] * df['Quantity']
    monetary = df.groupby('Customer ID')['TotalPrice'].sum().reset_index()
    monetary['Monetary'] = monetary['TotalPrice'] / df.groupby('Customer ID')['Invoice'].nunique().reset_index()['Invoice']
    df = pd.merge(df, monetary[['Customer ID', 'Monetary']], on='Customer ID', how='left')
    
    # Feature 5: Periodicity (P)
    df['InvoiceDateShift'] = df.groupby('Customer ID')['InvoiceDate'].shift(1)
    df['IVT'] = (df['InvoiceDate'] - df['InvoiceDateShift']).dt.days
    periodicity = df.groupby('Customer ID')['IVT'].std().reset_index()
    periodicity.columns = ['Customer ID', 'Periodicity']
    df = pd.merge(df, periodicity, on='Customer ID', how='left')
    
    # Seleccionamos las columnas que necesitamos
    result_df = df[['Customer ID', 'Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']].drop_duplicates(subset='Customer ID')
    
    return result_df

# Usamos FunctionTransformer para agregar custom_features al pipeline
custom_feature_transformer = FunctionTransformer(custom_features)

# Definir el pipeline numeric_transformations y categoric_transformations
numerical_columns = ['Price', 'Quantity']
categorical_columns = df_retail.columns.difference(numerical_columns).tolist()

numeric_transformations = Pipeline([
    ('IQR', IQR(lamb=1.5)),
    ('mean_imputer', SimpleImputer(strategy='mean'))
])

categoric_transformations = Pipeline([
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))
])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)],
                                       verbose_feature_names_out=False)

# Crear el retail_pipeline que encapsule el ColumnTransformer y las LRMFP
retail_pipeline = Pipeline([
    ('col_transformer', column_transformer),
    ('custom_features', custom_feature_transformer)
])

# Aplicar el pipeline
df_custom = retail_pipeline.fit_transform(df_retail)

# Mostrar las primeras filas del dataframe resultado
df_custom.head()

In [ ]:
# Explorar la distribución de las nuevas variables
explore_data(df_custom)


### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [32]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X):
        # Guardar los valores mínimos y máximos de cada columna en el dataframe X
        self.min_ = X.min()
        self.max_ = X.max()
        return self

    def transform(self, X):
        # Aplicar la transformación Min-Max para cada columna
        X_scaled = (X - self.min_) / (self.max_ - self.min_)
        return X_scaled

    def set_output(self, transform='default'):
        # No modificar este método, se usa para compatibilidad
        return self




In [ ]:
# Definir el pipeline numeric_transformations con el nuevo transformer MinMax
numeric_transformations = Pipeline([
    ('minmax_scaler', MinMax())
])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)],
                                       verbose_feature_names_out=False)

# Aplicamos ColumnTransformer a los datos
df_transformed = column_transformer.fit_transform(df_custom)

# Mostrar las primeras filas del dataframe transformado
df_transformed.head()

#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
# Definir el pipeline numeric_transformations con el nuevo transformer MinMax
numeric_transformations = Pipeline([
    ('IQR', IQR(lamb=1.5)),  # Paso para eliminar outliers
    ('mean_imputer', SimpleImputer(strategy='mean')),  # Imputación de la media
    ('minmax_scaler', MinMax())  # Escalado MinMax
])

# Pipeline para las variables categóricas
categoric_transformations = Pipeline([
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))  # Imputación de la moda
])

# Crear el ColumnTransformer actualizado
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)],
                                       verbose_feature_names_out=False)

# Aseguramos la salida en formato dataframe
column_transformer.set_output(transform='pandas')


In [ ]:
# Aplicamos ColumnTransformer a los datos
df_minmax = column_transformer.fit_transform(df_retail)
# Explorar la distribución de las variables en df_retail y df_minmax
explore_data(df_retail)   # Antes de la transformación
explore_data(df_minmax)   # Después de la transformación con Minm


### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

*Escriba su respuesta aquí*

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>